In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

In [5]:

def apertura_exploracion(csv):

    """ Función para leer csv, convertir a df y hacer una primera exploración.
    Igualar a variable con el nombre que quieres dar a DataFrame"""
    
    try:
        # Convertir el csv a DataFrame
        df = pd.read_csv(f"../data/{csv}.csv")        

        # Muestro las primeras filas
        display(df.head())

        # Obtengo las listas
        print(f"-----\n\nEl DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.\n-----")

        # Consulto si hay filas duplicadas
        print(f"\nEl número de filas duplicadas es {df.duplicated().sum()}\n-----")

        # Muestro el tipo de dato y si hay nulos por cada columna
        print("\nInformación del DataFrame:")
        df.info()

        # Muestro las estadísticas de columnas numéricas
        print("-----\n\nEstadísticas descriptivas:")
        display(df.describe().T)

        # Me devuelve un df que tendré que igualar a una variable
        return df  
                

    # Excepciones en caso de no encontrar el archivo o de que haya un error
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '../data/{csv}.csv'.")
        return None  
    
    except Exception as e:
        print(f"Ocurrió un error: {e}")
        return None 

In [6]:
df_movies = apertura_exploracion("raw_movies_dataset")

,Title,Genre,Year,Budget,Revenue,IMDB_Rating
0,The Last Journey,Drama,2020,11000000,3.404969e+07,4.9
1,Infinite Dreams,Drama,2004,178000000,5.504360e+08,6.6
2,Code of Shadows,Romance,2018,13000000,7.855773e+06,4.7
3,Ocean Whisper,Action,2016,148000000,1.769979e+08,7.2
4,Forgotten Realm,Romance,2017,61000000,1.264061e+08,NaN


-----

El DataFrame tiene 22 filas y 6 columnas.
-----

El número de filas duplicadas es 2
-----

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        22 non-null     object 
 1   Genre        18 non-null     object 
 2   Year         22 non-null     object 
 3   Budget       22 non-null     object 
 4   Revenue      19 non-null     float64
 5   IMDB_Rating  18 non-null     float64
dtypes: float64(2), object(4)
memory usage: 1.2+ KB
-----

Estadísticas descriptivas:


,count,mean,std,min,25%,50%,75%,max
Revenue,19.0,1.734479e+08,1.366885e+08,7.855773e+06,5.965693e+07,1.502366e+08,2.214591e+08,5.504360e+08
IMDB_Rating,18.0,5.883333e+00,1.487299e+00,3.900000e+00,4.900000e+00,5.550000e+00,6.700000e+00,9.100000e+00


In [ ]:

    

def eliminar_duplicados(df):
    """ Función para eliminar duplicados"""

    print(f"El df tiene {df.shape[0]} filas y {df.shape[0]} columnas.")
    print(f"Duplicados en df: {df.duplicated().sum()}")

    df = df.drop_duplicates()
    print("Eliminando filas duplicadas")

    print(f"Comprobamos que los duplicados se hayan eliminado. \nDuplicados en df: {df.duplicated().sum()}")

    return(f"El df tiene {df.shape[0]} filas y {df.shape[0]} columnas.")


def eliminar_irrelevantes(df, columna):
    """ Función para eliminar filas con datos irrelevantes"""
    print(f"El df tiene {df.shape[0]} filas y {df.shape[1]} columnas.")
    print(f"El df contiene {df[df[columna] > 0].shape[0]} filas y {df[df[columna] > 0].shape[1]} columnas con datos relevantes en la variable {columna}.")
    
    df = df[df[columna] > 0]
    print("Eliminando filas irrelevantes")
    print(f"El df tiene {df.shape[0]} filas y {df.shape[1]} columnas.")
    
    return df


def union_cvs(df1, df2):
    """ Función para unir ambos df y guardar en un nuevo csv.
    Igualar a una variable"""

    df3 = pd.merge(df1, df2, on="Loyalty Number", how="left")
    print("Uniendo los df")

    print(f"El nuevo df tiene {df3.shape[0]} filas y {df3.shape[1]} columnas.")

    df3.to_csv("files/Full Loyalty Program.csv", index=False)
    print("Guardando archivo como Full Loyalty Program.csv ")